## Using OpenCV to Improve Video Quality with CLAHE Transformation and Sharpening

In [1]:
import cv2
import numpy as np
import os

### Setting system paths for the project, existing labels, and CSV file for the existing labels

In [2]:
projectPath = '/home/motorns/Documents/work/hoffman/WhiteRabbit-Chrissy-2020-03-18'
labeledDataPath = 'labeled-data/chrissyproject'
csvLabels = 'CollectedData_Chrissy.csv'

### Changing directory for the project

In [3]:
os.chdir(projectPath)

In [4]:
inputVideo="videos/chrissyproject.mp4"
outputVideo="videos/chrissyprojectCLAHE.mp4"

In [5]:
cap = cv2.VideoCapture(inputVideo)

### Getting properties of the original video, which can be used to write the modified version

In [6]:
totalFrames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

### Setting outputfile to be of MP4 format

In [7]:
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
outVideo = cv2.VideoWriter(outputVideo, fourcc, fps, (width, height))

### Creating CLAHE object

In [8]:
clahe = cv2.createCLAHE(clipLimit=4.0, tileGridSize=(15,15))
sharpen = np.array(([0,-1,0],[-1,5,-1],[0,-1,0]),dtype="int")

### Main function to apply CLAHE transformation and sharpening

In [9]:
def claheSharpen(frame):
    frame_hsv = cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
    frame_hsv[:,:,2] = clahe.apply(frame_hsv[:,:,2])
    imEqCLAHE = cv2.cvtColor(frame_hsv, cv2.COLOR_HSV2BGR)
    sharpen_output = cv2.filter2D(imEqCLAHE,-1,sharpen)
    return sharpen_output

### Looping through the original video file, modifying each frame, and writing to the new mp4 (takes time to complete!)

In [10]:
while(cap.isOpened()):
    ret,frame = cap.read()
    if(ret==True):
        claheFrame = claheSharpen(frame)
        outVideo.write(claheFrame)
    else:
        break

In [11]:
outVideo.release()

## Applying CLAHE function to the Already Labeled Frames to Match the Video

In [12]:
import pandas as pd
import re

### Loading the labels CSV file

In [13]:
labeledFramesFile = pd.read_csv(os.path.join(labeledDataPath, csvLabels))

### Selecting filenames only and naming the column to imageName

In [14]:
labeledFramesPath = labeledFramesFile.scorer.reset_index().iloc[labeledFramesFile.index > 1]
labeledFramesPath = labeledFramesPath.scorer.rename("imageName").reset_index()

### Using regular expressions to pull out the original png file names and their 5-digit frame numbers

In [15]:
labeledFramesPath["imageName"] = labeledFramesPath.imageName.apply(lambda x: x.split("\\")[2])
labeledFramesPath["imageNumber"] = labeledFramesPath.imageName.apply(lambda x: int(re.search('\d{5}', x).group(0)))
labeledFramesPath.drop(["index"],axis=1,inplace=True)

### Looping over the list of frames, pulling from the video, applying CLAHE function, and writing to the Deeplabcut location

In [16]:
for index, name in labeledFramesPath.iterrows():
    cap.set(1,name.imageNumber);
    ret, frame = cap.read()
    claheFrame = claheSharpen(frame)
    cv2.imwrite(os.path.join(labeledDataPath, name.imageName),claheFrame)